# 12 - Realistic Projectile Motion - Modeling Air Resistance

We've all solved the ideal projectile motion problem neglecting air resistance. Let's explicitly cast this problem in terms of coupled, second-order differential equations so that we will be able to numerically analyze the scenario. The equations are given by 

\begin{equation*} \frac{\mathrm{d}x}{\mathrm{d}t}=v_x,\ \frac{\mathrm{d}y}{\mathrm{d}t}=v_y,\ \frac{\mathrm{d}v_x}{\mathrm{d}t}=0,\ \frac{\mathrm{d}v_y}{\mathrm{d}t}=-g \end{equation*}

using standard variable names.

Assumptions we've made:
* Neglect air resistance
    * Especially neglect varying atmospheric density
* Newtonian frame of ref (stationary ref frame)
* constant gravity equal to g - Actually has varying magnitude, direction will also change
* characteristics of projectile
* assumed flat surface of Earth #flatearthsociety


We can use computational methods to find a trajectory for this particle, plot the trajectory, and also generate an animation showing the motion of the projectile.

### Trajectory plot:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def f(r,t):
    x,y,vx,vy = r[0],r[1],r[2],r[3]
    fx = vx
    fy = vy
    fvx = 0
    fvy = -g
    return np.array([fx,fy,fvx,fvy],float)

g = 9.8
x0 = 0.0
y0 = 0.0
v0 = 20.0 # m/s
theta = 30.0 # degrees
vx0 = v0*np.cos(theta*np.pi/180)
vy0 = v0*np.sin(theta*np.pi/180)
t0 = 0.0     
tf = 10.0    
N = 1000
h = (tf-t0)/N 
 
# Data containers
tpoints = np.arange(t0,tf+h,h)
xpoints = []
ypoints = []

r = np.array([x0,y0,vx0,vy0],float)

for t in tpoints:
    xpoints.append(r[0])
    ypoints.append(r[1])
    k1 = h*f(r,t)
    k2 = h*f(r+0.5*k1,t+0.5*h)
    k3 = h*f(r+0.5*k2,t+0.5*h)
    k4 = h*f(r+k3,t+h)
    r += (k1+2*k2*2*k3+k4)/6
    if r[1] <= 0:
        r[1:4] = 0
        
print(np.max(xpoints))  
plt.plot(xpoints,ypoints)#,'k-',label='$y$ (m)')  

plt.xlabel('$x$ (m)')
plt.ylabel('$y$ (m)')
plt.title('Projectile motion trajectory')
plt.show()

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
**Pre-generated code:**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint

g = 9.8

def f(r,t):
    vecr = np.array([r[0],r[1]])
    vecv = np.array([r[2],r[3]])
        
    f = vecv
    fv = np.array([0,-g])
    
    return np.array([f[0],f[1],fv[0],fv[1]],float)


x0 = 0.0
y0 = 0.0
v0 = 20.0 # m/s
theta = 30.0 # degrees
vx0 = v0*np.cos(theta*np.pi/180)
vy0 = v0*np.sin(theta*np.pi/180)
t0 = 0.0     
tf = 5.0    
N = 1000
h = (tf-t0)/N 
 
tpoints = np.arange(t0,tf,h)
xpoints = []
ypoints = []
vxpoints = []
vypoints = []

# set up array with initial values x0,y0,vx0,vy0
r = np.array([x0,y0,vx0,vy0],float)

for t in tpoints:
    xpoints.append(r[0])
    ypoints.append(r[1])
    vxpoints.append(r[2])
    vypoints.append(r[3])
    # Below we've just changed `x` into `r`
    k1 = h*f(r,t)
    k2 = h*f(r+0.5*k1,t+0.5*h)
    k3 = h*f(r+0.5*k2,t+0.5*h)
    k4 = h*f(r+k3,t+h)
    r += (k1+2*k2+2*k3+k4)/6
    if r[1] <= 0:
        r[1]=0.0
        r[2] = 0.0
        r[3] = 0.0

plt.plot(xpoints,ypoints)#,'k-',label='$y$ (m)')  

plt.xlabel('$x$ (m)')
plt.ylabel('$y$ (m)')
plt.title('Projectile motion trajectory')
plt.show()

### Trajectory animation:

In [ ]:
# This code animates a moving object in the x-y plane based on data generated in the above program.

import vpython as vp     # get VPython modules for animation

scene = vp.canvas(center=vp.vector((xpoints[-1]-xpoints[0])/2,(ypoints[-1]-ypoints[0])/2,0)) # Pre-set the extent of the scene view so that full motion is centered

floor = vp.box(pos=vp.vector((xpoints[-1]-xpoints[0])/2,0,0), length=xpoints[-1], height=0.2, width=4)    # draw the floor 

ball = vp.sphere(pos=vp.vector(xpoints[0],ypoints[0],0), radius=1.0, color=vp.color.yellow, make_trail=True, interval=1, retain=50000) # Draw the ball

for i in range(np.size(xpoints)):
    vp.rate(100)
    ball.pos.x = xpoints[i] # Move the ball
    ball.pos.y = ypoints[i]


<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
## Let's make this more realistic. What about air resistance?
We know from our personal experiences that air resistance impedes motion. We experience this whenever we ride a bike or hold our hand out of a moving car. We also know that this force depends on the velocity of the object with respect to the air around it. We can model this behavior as 

\begin{equation*} \vec{F}_d = -b_1\vec{v} - b_2v\vec{v} . \end{equation*}

$\vec{F}_d$ is the drag force, $\vec{v}$ is the projectile velocity with respect to the medium, $v=|\vec{v}|$, and $b_1,b_2$ are linear and quadratic drag coefficients.

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
## Linear drag
Linear drag comes from the viscosity of the medium. The coefficient $b_1$ can be calculated from Stokes' law. Linear drag is often ignored because it is applicable to laminar (smooth) fluid flows. Most projectile motion will involve turbulent flows of the medium around the projectile. (SEE WANG EXERCISE 3.3)

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
## Quadratic drag
Quadratic drag is the dominant effect when flows become turbulent around an object moving through a medium. This is governed by the dimensionless Fluid Reynolds Number $Re=\rho v L/\mu$ where $\rho$ is the fluid density, $\mu$ is the fluid viscosity, $L$ is the characteristic length of the moving object, and $v$ is the velocity of the object moving through the fluid.

For a spherical object, experiments have shown that the transition to turbulence occurs at around $Re \ge 2300$. For a baseball, this transition occurs at a speed of $v\approx 0.5$ m/s.

We can develop an estimate for the quadratic coefficient. Assume an object (e.g. disc) with cross-sectional area $A$ moves through air with speed $v$ in time interval $\Delta t$. As it does so, it displaces the air mass in its path. The total displaced air is equal to the cylindrical volume $Av\Delta t$ multiplied by the air's mass density $\rho$. So, the mass of the displaced air is $\Delta m=\rho A v \Delta t$. If on average, the air mass $\Delta m$ is accelerated to speed $v$, the impulse $\Delta p=\Delta m v = \rho A v^2 \Delta t$. The force needed to accomplish this task is $F=\Delta p/\Delta t = \rho A v^2$. Remember, this is the force of the object on the air. By Newton's third law, this is equivalent to the magnitude of the force of the air on the object.

So, the quadratic drag coefficient $b_2$ is approximately $\rho A$. We can write this in a different way, letting $b_2=C_d \rho A /2$ so that 
\begin{equation*} \vec{F}_d \approx -b_2 v\vec{v} \approx -\frac{1}{2} C_d \rho A v \vec{v}\end{equation*} when the quadratic drag force is dominant. Here, $C_d$ is the quadratic drag coefficient and has an estimated value of 2.

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
### Model building and complexity
So, we've motivated the equation 
\begin{equation*} \vec{F}_d \approx -\frac{1}{2} C_d \rho A v \vec{v}.\end{equation*} There is still some physics that goes into the variable $C_d$. It is common to assume $C_d \sim 0.5$ for a spherical shape based on experiment. Our qualitative motivation above estimated $C_d=2$, but we should be happy that our qualitative argument even got us to within a factor of four; in the end, much of the essential physics was captured. 

We could stop here and be ok in most circumstances. In truth, though, $C_d$ is velocity dependent based on experimental evidence and depends on the roughness of the ball's surface. 

<img src="images/baseball_drag_coeff.jpg" alt="drawing" width="500"/>

Based on the figure, an empirical formula of the following form matches experimental evidence:

\begin{equation*}C_d = a+\frac{b}{1+\mathrm{exp}(\chi)}-c\times\left\{\begin{array}{lr}
        \mathrm{exp}(-\chi^2) & \text{for } \chi<0\\
        \mathrm{exp}(-\chi^2/4) & \text{for } \chi>0
        \end{array}\right.\end{equation*}
where
\begin{equation*} \chi = (v-v_c)/4,\ v_c=34,\ a=0.36,\ b=0.14,\ c=0.27.\end{equation*}
Here, speed $v$ is in m/s and $v_c$ is the critical speed.

Away from the critical speed $v_c$, the curve for $C_d$ appears to be pretty flat, so often the velocity dependence can be ignored to good approximation. If one wants higher precision, this more complicated behavior can be incorporated and will account for not just the quadratic drag, but also higher-order drag terms that go like $v^3,\ v^4,$ etc.

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
### Modeling quadratic drag
Model the following equations:

\begin{equation*} \frac{\mathrm{d}x}{\mathrm{d}t} = v_x,\ \frac{\mathrm{d}y}{\mathrm{d}t} = v_y, \\
\frac{\mathrm{d}v_x}{\mathrm{d}t} = -\frac{b_2}{m}vv_x,\ \frac{\mathrm{d} v_y}{\mathrm{d}t}=-g-\frac{b_2}{m} vv_y, \end{equation*}

where $v=\sqrt{v_x^2+v_y^2}$.

### Code (starting with $C_D=0.5$ initially, then generalizing):

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def f(r,t):
    x = r[0]
    y = r[1]
    vx = r[2]
    vy = r[3]
    v = np.sqrt(vx**2+vy**2)
    
    b2 = Cd*rho*A/2
    fx = vx
    fy = vy
    fvx = -b2/m*v*vx
    fvy = -g - b2/m*v*vy

    return np.array([fx,fy,fvx,fvy],float)

m = .145 # kg
rho = 1.225 # kg/m^3
radius = .0365 # m
A = np.pi*radius**2
Cd = 0.5
g = 9.8
x0 = 0.0
y0 = 0.0
v0 = 20.0 # m/s
theta = 30.0 # degrees
vx0 = v0*np.cos(theta*np.pi/180)
vy0 = v0*np.sin(theta*np.pi/180)
t0 = 0.0     
tf = 5.0    
N = 1000
h = (tf-t0)/N 
 
tpoints = np.arange(t0,tf,h)
xpoints = []
ypoints = []
vxpoints = []
vypoints = []

# set up array with initial values x0,y0,vx0,vy0
r = np.array([x0,y0,vx0,vy0],float)

for t in tpoints:
    xpoints.append(r[0])
    ypoints.append(r[1])
    vxpoints.append(r[2])
    vypoints.append(r[3])
    # Below we've just changed `x` into `r`
    k1 = h*f(r,t)
    k2 = h*f(r+0.5*k1,t+0.5*h)
    k3 = h*f(r+0.5*k2,t+0.5*h)
    k4 = h*f(r+k3,t+h)
    r += (k1+2*k2+2*k3+k4)/6
    if r[1] <= 0:
        r[1]=0.0
        r[2] = 0.0
        r[3] = 0.0

plt.plot(xpoints,ypoints)#,'k-',label='$y$ (m)')  

plt.xlabel('$x$ (m)')
plt.ylabel('$y$ (m)')
plt.title('Projectile motion trajectory')
plt.show()

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
**Pre-generated code:**
\begin{equation*}C_d = a+\frac{b}{1+\mathrm{exp}(\chi)}-c\times\left\{\begin{array}{lr}
        \mathrm{exp}(-\chi^2) & \text{for } \chi<0\\
        \mathrm{exp}(-\chi^2/4) & \text{for } \chi>0
        \end{array}\right.\end{equation*}
\begin{equation*} \chi = (v-v_c)/4,\ v_c=34,\ a=0.36,\ b=0.14,\ c=0.27.\end{equation*}

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint

g = 9.8
m = .145 # kg
rho = 1.225 # kg/m^3
radius = .0365 # m
A = np.pi*radius**2

def f(r,t,dragq):
    vecr = np.array([r[0],r[1]])
    vecv = np.array([r[2],r[3]])
    vnorm = np.linalg.norm(vecv)
    ######## following lines only valid for baseball! #########
    chi = (vnorm-34.0)/4.0
    if chi <= 0:
        cc = np.exp(-chi**2)
    else:
        cc = np.exp(-chi**2/4)
    
    if dragq == 0:
        Cd = 0
    elif dragq == 1:
        Cd = 0.5
    elif dragq == 2:
        Cd = .36 + .14/(1+np.exp(chi))-0.27*cc
    elif dragq == 3:
        Cd = 0.36
    ##########################################################
    b2 = 0.5*Cd*rho*A
    
    f = vecv
    fv = np.array([0,-g])-(b2/m)*vnorm*vecv
    
    return np.array([f[0],f[1],fv[0],fv[1]],float)




x0 = 0.0
y0 = 0.0
v0 = 100.0 # m/s
theta = 30.0 # degrees
vx0 = v0*np.cos(theta*np.pi/180)
vy0 = v0*np.sin(theta*np.pi/180)
t0 = 0.0     
tf = 5.0    
N = 1000
h = (tf-t0)/N 

tpoints = np.arange(t0,tf+h,h)

for i in range(4):
    dragq = i
    xpoints = []
    ypoints = []
    vxpoints = []
    vypoints = []
    # set up array with initial values x0,y0
    r = np.array([x0,y0,vx0,vy0],float)
    for t in tpoints:
        xpoints.append(r[0])
        ypoints.append(r[1])
        vxpoints.append(r[2])
        vypoints.append(r[3])
        # Below we've just changed `x` into `r`
        k1 = h*f(r,t,dragq)
        k2 = h*f(r+0.5*k1,t+0.5*h,dragq)
        k3 = h*f(r+0.5*k2,t+0.5*h,dragq)
        k4 = h*f(r+k3,t+h,dragq)
        r += (k1+2*k2+2*k3+k4)/6
        if r[1] <= 0:
            r[1]=0.0
            r[2] = 0.0
            r[3] = 0.0
    
    #plt.figure(dpi=100)            
    plt.plot(xpoints,ypoints)#,'k-',label='$y$ (m)')  

plt.xlabel('$x$ (m)')
plt.ylabel('$y$ (m)')
plt.title('Projectile motion trajectory')
plt.legend(('ideal','$C_D=0.5$','$C_D(v)$','C_d=0.38'),loc='best')
plt.show()

<br><br><br><br><br><br><br><br><br><br><br><br><br><br>
## Magnus force
How does one throw a curveball in baseball? Or how does one 'bend' a kick in soccer (football)? The spin of an object as it interacts with the fluid (air) around it can cause additional forces beyond just drag. One example is the Magnus force. See the videos ([Basketball over a dam](https://youtu.be/QtP_bh2lMXc) and [Flying Paper Tube](https://www.youtube.com/watch?v=O6JdMSpiIqU)) for demonstrations of the Magnus force.

The Magnus force is given by

\begin{equation*} \vec{F}_{magnus} = \alpha \vec{\omega}\times\vec{v} \end{equation*}

where $\alpha$ is a constant with the dimensions of mass, $\omega$ is the angular speed of the rotating object (in a direction along the axis of rotation and given by the right hand rule), and $v$ is the object's linear speed with respect to the air. For a sphere, the equation takes on the form

\begin{equation*}\vec{F}_{magnus} = \frac{1}{2}C_L \rho A \frac{v}{\omega}\vec{\omega}\times\vec{v},\end{equation*}

where $C_L$ is called the lift coefficient. The lift coefficient has been experimentally shown to be mainly a function of theso-called spin parameter $S = r\omega/v$, so we have

\begin{equation*}F_{magnus} = \frac{1}{2} C_L \frac{\rho A r}{S}.\end{equation*}

**Code to show rotation axis definition:**

In [ ]:
import vpython as vp
import numpy as np

scene = vp.canvas()
N = 1000
omega = vp.vector(0,20,0)
ball = vp.sphere(pos=vp.vector(0,0,0),radius=.5, texture=vp.textures.rough)
spin = vp.arrow(axis=omega,pos=vp.vector(0,0,0),length=2)


phi = np.pi/128
for i in range(N):
    vp.rate(40)
    spin.rotate(angle=phi), ball.rotate(angle=phi,axis=omega)  #spin
    

<br><br><br><br><br><br><br><br><br><br><br><br><br>
## Example: Model the motion of a baseball, including both $\vec{F}_d$ and $\vec{F}_{magnus}$.

For a baseball, $C_L = 0.62 \times S^{0.7}$ from experiment. So, the equations that we must model are given by

\begin{equation*} \frac{\mathrm{d}\vec{r}}{\mathrm{d}t}=\vec{v},\quad \frac{\mathrm{d}\vec{v}}{\mathrm{d}t}=-g\hat{j}-\frac{b_2}{m}v\vec{v} + \frac{\alpha}{m}\vec{\omega}\times \vec{v} \end{equation*}

where $\vec{r}$ is the position vector and the $x$ direction refers to forward motion, $y$ refers to vertical motion, and $z$ refers to sideways motion. This is a set of six first-order ODEs to be solved.

In [3]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint

g = 9.8
m = .145 # kg
rho = 1.225 # kg/m^3
radius = .0365 # m
A = np.pi*radius**2
omega = np.array([0,0,-200])


def f(r,t,dragq,magnusq):
    vecr = np.array([r[0],r[1],r[2]])
    vecv = np.array([r[3],r[4],r[5]])
    vnorm = np.linalg.norm(vecv)
    ######## following lines only valid for baseball! #########
    chi = (vnorm-34.0)/4.0
    
    if chi <= 0:
        cc = np.exp(-chi**2)
    else:
        cc = np.exp(-chi**2/4)
    
    if dragq == 0:
        Cd = 0
    elif dragq == 1:
        Cd = 0.5
    elif dragq == 2:
        Cd = .36 + .14/(1+np.exp(chi))-0.27*cc
    elif dragq == 3:
        Cd = 0.36

    b2 = 0.5*Cd*rho*A
    ##########################################################
    
    S = radius*np.linalg.norm(omega)/vnorm
    CL = 0.62 * S**(0.7) # For baseball only!
    alpha = 0.5 * CL * rho * A * radius / S
    if magnusq == 0:
        dvdt_magnus = np.zeros(3)
    else:
        dvdt_magnus = (alpha/m)*np.cross(omega,vecv)
    
    f = vecv
    fv = np.array([0,-g,0])-(b2/m)*vnorm*vecv+dvdt_magnus
    
    return f[0],f[1],f[2],fv[0],fv[1],fv[2]

x0 = 0.0
y0 = 2.0
z0 = 0.0
v0 = 30.0 # m/s
theta = 0.0 # degrees
vx0 = v0*np.cos(theta*np.pi/180)#10.0
vy0 = v0*np.sin(theta*np.pi/180)#10.0
vz0 = 0
t0 = 0.0     
tf = 0.7    
N = 1000
h = (tf-t0)/N 

R = 18.4 # It is 60'6", or 18.4 meters from pitcher to home plate

tpoints = np.arange(t0,tf,h)

# set up array with initial values x0,y0
r = np.array([x0,y0,z0,vx0,vy0,vz0],float)

for i in range(3):
    dragq = i
    magnusq = 1
    rpoints = odeint(f,r,tpoints,args=(dragq,magnusq,)) 
    
    xpoints = rpoints[rpoints[:,0]<R,0]
    ypoints = rpoints[rpoints[:,0]<R,1]
    zpoints = rpoints[rpoints[:,0]<R,2]
    #ypoints[ypoints<=0]=0
    
    #plt.figure(dpi=100)            
    #plt.plot(xpoints,ypoints)#,'k-',label='$y$ (m)')  

#plt.xlabel('$x$ (m)')
#plt.ylabel('$y$ (m)')
#plt.title('Projectile motion trajectory, x-y plane')
#plt.legend(('ideal','$C_D=0.5$','$C_D(v)$'),loc='best')
#plt.show()

#tpoints = tpoints[rpoints[:,0]<R]
#plt.plot(xpoints,zpoints)
#plt.title('Projectile motion, x-z plane')
#plt.xlabel('$x$ (m)')
#plt.ylabel('$z$ (m)')
#plt.show()

In [ ]:
import vpython as vp     # get VPython modules for animation
scene = vp.canvas(center=vp.vector((xpoints[-1]-xpoints[0])/2,(ypoints[-1]-ypoints[0])/2,0)) # Needed to start a new Vpython display for the current code cell

floor = vp.box(pos=vp.vector((xpoints[-1]-xpoints[0])/2,0,0), length=xpoints[-1], height=0.2, width=4)    # floor 

ball = vp.sphere(pos=vp.vector(xpoints[0],ypoints[0],zpoints[0]), radius=0.1, color=vp.color.yellow, make_trail=True,
              interval=1, retain=50000) # ball  

for i in range(np.size(xpoints)):
    vp.rate(100)
    ball.pos.x = xpoints[i]
    ball.pos.y = ypoints[i]
    ball.pos.z = zpoints[i]

In [4]:
import vpython as vp

# Establish the vpython view and objects
scene = vp.canvas(background=vp.vector(.5,.5,1), forward=vp.vector(-1,-.1,-.1),center=vp.vector(.5*R,1,0)) # Set up screen view
floor = vp.box(pos=vp.vector(R/2,0,0), length=1.1*R, height=.1, width=8, color=vp.color.orange, opacity=0.7) # Create the ground
zone = vp.curve(pos=[vp.vector(R,0,1),vp.vector(R,1,1),vp.vector(R,1,-1),vp.vector(R,0,-1)], radius=0.02) # Generate zone to aim for
ball = vp.sphere(pos=vp.vector(0,0,0),radius=.2, texture=vp.textures.rough) # Generate ball with rough texture
trail = vp.curve(pos=vp.vector(0,0,0),radius=0.04) # parameters for the trail that the ball will leave
ideal = vp.curve(pos=vp.vector(0,0,0),radius=0.04,color=vp.color.green) # parameters for trail for the ideal (no drag or magnus) case
spin = vp.arrow(axis=vp.vector(omega[0],omega[1],omega[2]),pos=vp.vector(0,0,0),length=1) # Create an arrow showing spin axis.

# Advance animation through full motion
offset = 0.4*vp.vector(omega[0],omega[1],omega[2])/np.linalg.norm(omega) # Used to center spin axis arrow on center of ball
phi = np.pi/64 # How much do the ball and arrow rotate per frame
for i in range(np.size(xpoints)):
    vp.rate(80)
    if xpoints[i] <= R and ypoints[i] >=0: # Advance ball if it is above the ground and hasn't reached home plate
        ball.pos = vp.vector(xpoints[i],ypoints[i],zpoints[i]) # move ball 
        spin.pos = vp.vector(xpoints[i],ypoints[i],zpoints[i]) -offset # Move arrow
        spin.rotate(angle=phi) # Spin the arrow
        ball.rotate(angle=phi,axis=vp.vector(omega[0],omega[1],omega[2]))  # Spin the ball
        trail.append(pos=ball.pos) # Extend the trail left by the ball
    
    xideal = x0 + vx0*tpoints[i] # Calculate x position assuming no drag or Magnus
    yideal = y0+vy0*tpoints[i]-0.5*g*tpoints[i]*tpoints[i] # Ditto
    zideal = 0. # Also ditto
    if xideal <= R and yideal >= 0: # Generate ideal trail as long as it remains above ground and hasn't reached home plate
        ideal.append(pos=(xideal,yideal,zideal))  # ideal case

<IPython.core.display.Javascript object>